In [1]:
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the POSTGRES_PWD variable
POSTGRES_PWD = os.getenv('POSTGRES_PWD')

# Connect to PostgreSQL server
conn = psycopg2.connect(
    host="localhost",  # Update with your database host
    database="LysozomeInWater",
    user="postgres",
    password=POSTGRES_PWD,
    port='5433'
)

# Create a cursor object
cur = conn.cursor()

# Execute a simple query to verify connection
cur.execute("SELECT current_database();")
db_name = cur.fetchone()
print(f"Connected to database: {db_name[0]}")





Connected to database: LysozomeInWater


In [2]:
# Drop existing tables if they exist
cur.execute("DROP TABLE IF EXISTS Trajectory")
cur.execute("DROP TABLE IF EXISTS Atoms")

# Create Atoms table
create_atoms_table_query = """
CREATE TABLE Atoms (
    AtomNum INTEGER PRIMARY KEY,
    Atom VARCHAR(10),
    ResidueGroup VARCHAR (10),
    ResidueString VARCHAR(10)
);
"""

# Create Trajectory table
create_trajectory_table_query = """
CREATE TABLE Trajectory (
    AtomNum INTEGER,
    TimeFrame FLOAT,
    x FLOAT,
    y FLOAT,
    z FLOAT,
    PRIMARY KEY (AtomNum, TimeFrame),
    FOREIGN KEY (AtomNum) REFERENCES Atoms(AtomNum)
);
"""

# Execute the queries
cur.execute(create_atoms_table_query)
cur.execute(create_trajectory_table_query)
conn.commit()


In [3]:
# Parse trajectory file

import re

def parse_trajectory(file_path):
    data = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        time_frame = None

        for line in lines:
            # Skip lines with just the molecule count (e.g., "1960")
            if line.strip().isdigit():
                continue
            # Skip lines with just the simulation box dimensions (e.g., 10.0 10.0 10.0)
            elif re.match(r'^\s*(\d+\.\d+\s*){3}$', line.strip()):
                continue
            # Extract time frame using regex
            time_frame_match = re.search(r't=\s*([\d\.]+)', line)
            if time_frame_match:
                time_frame = float(time_frame_match.group(1))
            else:  # Atom data line
                fields = line.split()
                atom_number = int(fields[2])
                atom = fields[1]
                residue_goup = fields[0]
                residue_string = re.sub(r'^\d+', '', residue_goup)  # Remove leading digits
                x, y, z = map(float, fields[3:])
                data.append((atom_number, atom, residue_goup, residue_string, time_frame, x, y, z))
    return data



In [4]:
def insert_atoms(data, conn, cur):
    insert_query = """
    INSERT INTO atoms (AtomNum, Atom, ResidueGroup, ResidueString)
    VALUES (%s, %s, %s, %s)
    ON CONFLICT (AtomNum) DO NOTHING;  -- Avoid duplicate entries
    """
    cur.executemany(insert_query, data)
    conn.commit()

def insert_trajectory(data, conn, cur):
    insert_query = """
    INSERT INTO trajectory (AtomNum, TimeFrame, x, y, z)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (AtomNum, TimeFrame) DO NOTHING;  -- Avoid duplicate entries
    """
    cur.executemany(insert_query, data)
    conn.commit()


In [ ]:
from pathlib import Path

# Parse file and insert data
filename = 'protein_output.txt'
traj_filepath = Path.cwd() / filename
data = parse_trajectory(file_path=traj_filepath)

# Extract atom-specific data from total trajectory data
atoms_data = [(AtomNum, Atom, ResidueGroup, ResidueString) for (AtomNum, Atom, ResidueGroup, ResidueString, *rest) in data]
trajectory_data = [(AtomNum, TimeFrame, x, y, z) for (AtomNum, _, _, _, TimeFrame, x, y, z) in data]

insert_atoms(atoms_data, conn, cur)
insert_trajectory(trajectory_data, conn, cur)

/Users/vibhavlaud/Desktop/cs596/final_project


In [6]:
# Close the cursor and connection
cur.close()
conn.close()